## Trying to create the MDI with the RPKM adjusted gene values, let's see if it lines up!!!

In [5]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [6]:
df = pd.read_csv('GSE99622_hanamsagar2017_tpm_unmelted_v2.csv', index_col = 0, header = [0, 1, 2, 3])

In [7]:
### (expression value of the gene in a sample—minimum expression for the gene across all samples)/(maximum expression for the gene across all
### samples—minimum expression for the gene across all samples),
### scale all values so that they add equal weight to the index

def scale_expression(df):
    scaled_expression_df = df.copy()

    for row in range(len(scaled_expression_df)):
        scaled_expression_df.iloc[row] = (scaled_expression_df.iloc[row] - np.min(scaled_expression_df.iloc[row])) / np.max(scaled_expression_df.iloc[row] - np.min(scaled_expression_df.iloc[row]))
        
        
    return scaled_expression_df

In [8]:
scaled_df = scale_expression(df)

In [9]:
scaled_df

E18                                     P14            \
            Female      Male    Female      Male    Female             
               SAL       SAL       SAL       SAL       SAL             
           F_E18 1   M_E18 1   F_E18 3   M_E18 4   F_P14 1   F_P14 2   
gene                                                                   
Snx7      0.588140  0.368395  0.568197  1.000000  0.181462  0.219422   
Dnajc18   0.797903  0.653250  0.657849  0.000000  0.111909  0.408285   
Ipo11     0.853112  0.442486  0.719860  0.000000  0.208482  0.398846   
Fam101b   0.416253  0.125174  0.209573  0.073500  0.270525  0.357042   
Mgrn1     0.692032  0.705919  0.782617  0.084909  1.000000  0.751368   
...            ...       ...       ...       ...       ...       ...   
Prune     0.942675  0.624474  0.990320  0.543005  0.260842  0.402808   
Wdr45     0.607076  0.798438  0.527484  0.548389  0.361703  0.347226   
Cdkn2aip  0.747043  0.552209  0.538767  0.000000  0.155256  0.379097   
Phf20     0.852279  0.625225  0.638040  0.000000  0.496946  0.497212   
Mettl2    0.732807  0.692819  0.577084  0.000000  0.255052  0.332333   

                                                  ...   P60 + LPS              \
                                                  ...      Female        Male   
                                                  ...         LPS         LPS   
           F_P14 3   F_P14 4   F_P14 5   F_P14 6  ... F_P60_LPS 3 M_P60_LPS 3   
gene                                              ...                           
Snx7      0.346333  0.241872  0.676114  0.452090  ...    0.122795    0.154882   
Dnajc18   0.629233  0.236698  0.488088  0.484321  ...    0.431513    0.262827   
Ipo11     0.636584  0.382300  0.502430  0.789239  ...    0.173647    0.491490   
Fam101b   0.532354  0.406182  0.710827  0.655118  ...    0.061817    0.018229   
Mgrn1     0.772810  0.867624  0.641571  0.494001  ...    0.781737    0.400948   
...            ...       ...       ...       ...  ...         ...         ...   
Prune     0.337056  0.311923  0.239790  0.524430  ...    0.238199    0.232919   
Wdr45     0.429304  0.205366  0.331340  0.118872  ...    0.531592    0.147962   
Cdkn2aip  0.334960  0.341667  0.249145  0.338652  ...    0.640671    0.409028   
Phf20     0.461477  0.410191  0.361023  0.230698  ...    0.652753    0.815504   
Mettl2    0.308777  0.342049  0.053049  0.123743  ...    0.551175    0.114495   

                                                     P60              \
                                                    Male               
                                                     SAL               
         M_P60_LPS 4 M_P60_LPS 5 M_P60_LPS 6 M_P60_Sal 1 M_P60_Sal 2   
gene                                                                   
Snx7        0.282375    0.196553    0.160842    0.294345    0.094570   
Dnajc18     0.199129    0.218737    0.239025    0.592543    0.385785   
Ipo11       0.231167    0.098126    0.427983    0.644599    0.334307   
Fam101b     0.071497    0.003972    0.000000    0.214608    0.079144   
Mgrn1       0.493208    0.749388    0.644851    0.487874    0.846888   
...              ...         ...         ...         ...         ...   
Prune       0.239081    0.140453    0.389822    0.341080    0.362487   
Wdr45       0.000000    0.293499    0.379687    0.378595    0.350806   
Cdkn2aip    0.205182    0.376656    0.283741    0.372469    0.684693   
Phf20       0.409657    0.394104    0.788039    0.723134    0.650915   
Mettl2      0.212880    0.097181    0.106913    0.380824    0.254098   

                                              
                                      Female  
                                         SAL  
         M_P60_Sal 3 M_P60_Sal 4 F_P60_Sal 6  
gene                                          
Snx7        0.218336    0.350578    0.467373  
Dnajc18     0.979729    0.861057    0.033983  
Ipo11       0.572597    0.475738    0.543234  
Fam101b     0.328676    0.325070    

In [10]:
def identify_significant_genes(df):
    
    e18_all= df['E18']
    p60 = df['P60']
    
    p60_all = p60.iloc[:, p60.columns.get_level_values(1)=='SAL']

    pvals = np.zeros(shape = len(df))
    sig = np.zeros(shape = len(df))
    logdiff = np.zeros(shape = len(df))

    for row in range(len(df)):
        pvals[row] = stats.ttest_ind(e18_all.iloc[row], p60_all.iloc[row])[1]
        sig[row] = stats.ttest_ind(e18_all.iloc[row], p60_all.iloc[row])[0]
        logdiff[row] = np.log2(np.mean(p60_all.iloc[row])/np.mean(e18_all.iloc[row]))
    df['pvals'] = pvals
    df['sig'] = sig
    df['logdiff'] = logdiff

    direction = [0] * len(df)
    for row in range(len(df)):
        if df['pvals'][row] < 0.05:
            if df['logdiff'][row] > 0:
                direction[row] = 'UP'
            else:
                direction[row] = 'DOWN'
        else:
            direction[row] = 'N/A'
    df['direction'] = direction
    

    return df

In [11]:
new_df = identify_significant_genes(scaled_df)

In [12]:
def remove_unsignificant_rows(df):
    df.reset_index(inplace = True)
    for row in range(len(df)):
        if df['direction'][row] == 'N/A':
            df.drop(row, inplace = True)  
    df.reset_index(drop=True, inplace=True)
    return df

In [13]:
df_cleaned = remove_unsignificant_rows(new_df)

In [14]:
down_genes = df_cleaned['gene'][df_cleaned['direction'] == 'DOWN'].to_list()
up_genes = df_cleaned['gene'][df_cleaned['direction'] == 'UP'].to_list()

In [15]:
def generate_index(df):
    
    samples = df.columns[1:-4]
    
    index_per_sample = [0] * len(samples)
    i = -1
    
    for sample in samples:
        i = i + 1
        index_per_sample[i] = np.mean(df[sample][df['direction'] == 'UP']) / np.mean(df[sample][df['direction'] == 'DOWN'])
        
    return index_per_sample

In [16]:
final_df = pd.DataFrame([generate_index(df_cleaned)], columns = df_cleaned.columns[1:-4])
final_df

E18                                     P14                      \
     Female      Male    Female      Male    Female                       
        SAL       SAL       SAL       SAL       SAL                       
    F_E18 1   M_E18 1   F_E18 3   M_E18 4   F_P14 1   F_P14 2   F_P14 3   
0  0.449358  0.283308  0.447488  0.093992  1.370898  1.392601  1.214291   

                                 ...   P60 + LPS                          \
                                 ...      Female        Male               
                                 ...         LPS         LPS               
    F_P14 4   F_P14 5   F_P14 6  ... F_P60_LPS 3 M_P60_LPS 3 M_P60_LPS 4   
0  1.392616  0.972999  1.307666  ...    2.249739    2.954748    2.102531   

                                  P60                                      \
                                 Male                                       
                                  SAL                                       
  M_P60_LPS 5 M_P60_LPS 6 M_P60_Sal 1 M_P60_Sal 2 M_P60_Sal 3 M_P60_Sal 4   
0    2.161509     2.58062    2.069634    2.284102    1.593021    1.774172   

               
       Female  
          SAL  
  F_P60_Sal 6  
0    1.713373  

[1 rows x 60 columns]

In [17]:
## scale data to between 0 and 1
final_df_scaled = pd.DataFrame(final_df.iloc[0] - np.min(final_df.iloc[0])) / (np.max(final_df.iloc[0] - np.min(final_df.iloc[0])))
final_df_scaled.reset_index(inplace = True)

In [18]:
final_df_scaled.columns = ['age', 'sex', 'tx', 'sample_name', 'index']

In [19]:
final_df_scaled.to_csv('C:\\Users\\Ben\\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\mdi\\tpm_data_index_v2.csv')

In [20]:
regulated_genes = pd.DataFrame((dict([ (k,pd.Series(v)) for k,v in {'gene' : df_cleaned['index'], 'direction' : df_cleaned['direction'], 'valence' : df_cleaned['logdiff']}.items() ])))

regulated_genes.to_csv('C:\\Users\\Ben\\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\mdi\\tpm_data_index_gene_list.csv')

KeyError: 'index'